## Tensorflow DNNClassifer

- Data are stored in pickle by prepare_data sript

### Load packages

In [1]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt

### Load data

In [2]:
## load data and scalled it
with open('../data/features.pickle', 'rb') as handle:
    d = pickle.load(handle)
    
d['train']['features'][np.isnan(d['train']['features'])] = 0
d['train']['features'] = scale( d['train']['features'], axis=0, with_mean=True, with_std=True, copy=True )

d['validation']['features'][np.isnan(d['validation']['features'])] = 0
d['validation']['features'] = scale( d['validation']['features'], axis=0, with_mean=True, with_std=True, copy=True )


d['test']['features'][np.isnan(d['test']['features'])] = 0
d['test']['features'] =scale(d['test']['features'], axis=0, with_mean=True, with_std=True, copy=True)

## covert into dummy values
d.keys()

['test', 'train', 'validation']

### DNNClassifer analysis by tensorflow

In [3]:
### Tensorflow DNN
feature_columns = [tf.feature_column.numeric_column("x",shape=[1,188])]

# Build 2 layer DNN classifier
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[100,100,100],
    optimizer=tf.train.AdamOptimizer(1e-4),
    n_classes=3,
    dropout=0.1,
    model_dir="./tmp/dnn_model"
)

# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": d['train']['features']},
    y=d['train']['label'],
    num_epochs=None,
    batch_size=200,
    shuffle=True
)

classifier.train(input_fn=train_input_fn, steps=10000)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a1707e110>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': './tmp/dnn_model', '_global_id_in_cluster': 0, '_save_summary_steps': 100}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./tmp/dnn_model/model.ckpt.
INFO:tensorflow:loss = 230.001, 

INFO:tensorflow:loss = 125.51, step = 7501 (0.693 sec)
INFO:tensorflow:global_step/sec: 150.715
INFO:tensorflow:loss = 109.307, step = 7601 (0.665 sec)
INFO:tensorflow:global_step/sec: 154.759
INFO:tensorflow:loss = 93.4602, step = 7701 (0.644 sec)
INFO:tensorflow:global_step/sec: 149.675
INFO:tensorflow:loss = 103.6, step = 7801 (0.669 sec)
INFO:tensorflow:global_step/sec: 149.41
INFO:tensorflow:loss = 113.559, step = 7901 (0.668 sec)
INFO:tensorflow:global_step/sec: 143.012
INFO:tensorflow:loss = 92.0994, step = 8001 (0.701 sec)
INFO:tensorflow:global_step/sec: 154.721
INFO:tensorflow:loss = 126.311, step = 8101 (0.646 sec)
INFO:tensorflow:global_step/sec: 136.006
INFO:tensorflow:loss = 110.187, step = 8201 (0.734 sec)
INFO:tensorflow:global_step/sec: 157.916
INFO:tensorflow:loss = 107.968, step = 8301 (0.633 sec)
INFO:tensorflow:global_step/sec: 145.624
INFO:tensorflow:loss = 98.2706, step = 8401 (0.688 sec)
INFO:tensorflow:global_step/sec: 142.038
INFO:tensorflow:loss = 113.726, st

### Accuracy on the validation dataset

In [4]:
# Define the test inputs
validation_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": d['validation']['features']},
    y=d['validation']['label'],
    num_epochs=1,
    shuffle=False
)

# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=validation_input_fn)["accuracy"]
print("\nValidation Accuracy: {0:f}%\n".format(accuracy_score*100))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-26-00:10:44
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/dnn_model/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-26-00:10:45
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.790657, average_loss = 0.526593, global_step = 10000, loss = 67.268

Validation Accuracy: 79.065657%



### Output prediction on the testing dataset to be evaluated by drivendata.org

In [5]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": d['test']['features']},
    shuffle=False
)

## Output prediction for test, evalucate it through https://www.drivendata.org/
pred = [p['class_ids'][0]  for p in classifier.predict(input_fn=test_input_fn)]
mapping = dict(enumerate(d['train']['label_original'].cat.categories))
pred_label = [mapping[i] for i in pred ]

outd = pd.DataFrame({"id":d['test']["id"]['id'].tolist(),"status_group": pred_label})

outd.to_csv(path_or_buf="../result/TF-DNN-predctions.csv",index=False)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/dnn_model/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


### Output package versions used

In [6]:
#find the names of the imported modules
import types
def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__

#exclude all modules not listed by `!pip freeze`
excludes = ['__builtin__', 'types', 'IPython.core.shadowns', 'sys', 'os']
imported_modules = [module for module in imports() if module not in excludes]
pip_modules = !pip freeze #you could also use `!conda list` with anaconda

#print the names and versions of the imported modules
for module in pip_modules:
    name, version = module.split('==')
    if name in imported_modules:
        print(name + '\t' + version)

numpy	1.13.3
pandas	0.20.3
tensorflow	1.8.0
